In [16]:
from Py6S import *
import numpy as np

In [17]:
# Parameters
wind_speed = 1
wind_azimuth = 0
salinity = 34.3
pigmentation = 0.3

# Boundary Parameters
max_outgoing_zenith = 89
resolution = 25

In [18]:
s = SixS()
s.wavelength = Wavelength(2.2)
s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.NoGaseousAbsorption)
s.aero_profile = AeroProfile.PredefinedType(AeroProfile.NoAerosols)
s.ground_reflectance = GroundReflectance.HomogeneousOcean(wind_speed, wind_azimuth, salinity, pigmentation)

In [19]:
# Run 6S for a variety of geometries
outgoing_azimuth = 180
outgoing_zeniths = np.linspace(0, np.deg2rad(max_outgoing_zenith), max_outgoing_zenith)    


incoming_azimuth = 0
incoming_zeniths = np.linspace(0, np.deg2rad(89), 90)

In [20]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

file_names = []

# Loop over tuple of outgoing azimuths and zeniths
idx = 0
max_idx = len(incoming_zeniths)
for incoming_zenith in incoming_zeniths:
    file_name = f"reflectance_{incoming_zenith}.json"
    file_names.append(file_name)
    reflectances = {}
    def process(outgoing_zenith):
        s.geometry = Geometry.User()
        s.geometry.solar_z = np.rad2deg(incoming_zenith)
        s.geometry.solar_a = np.rad2deg(incoming_azimuth)
        s.geometry.view_z = np.rad2deg(outgoing_zenith)
        s.geometry.view_a = np.rad2deg(outgoing_azimuth)
        s.run()
        return outgoing_zenith, s.outputs.values['apparent_reflectance']

    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(process, outgoing_zenith) for outgoing_zenith in outgoing_zeniths]

        for future in as_completed(futures):
            outgoing_zenith, reflectance = future.result()
            reflectances[outgoing_zenith] = reflectance

    # Save dataset as JSON to file in the 'data' directory
    with open(f'data/{file_name}', 'w') as f:
        json.dump(reflectances, f)

    idx += 1
    print(f"Progress: {idx}/{max_idx}")


Progress: 1/90
Progress: 2/90
Progress: 3/90
Progress: 4/90
Progress: 5/90
Progress: 6/90
Progress: 7/90
Progress: 8/90
Progress: 9/90
Progress: 10/90
Progress: 11/90
Progress: 12/90
Progress: 13/90
Progress: 14/90
Progress: 15/90
Progress: 16/90
Progress: 17/90
Progress: 18/90
Progress: 19/90
Progress: 20/90
Progress: 21/90
Progress: 22/90
Progress: 23/90
Progress: 24/90
Progress: 25/90
Progress: 26/90
Progress: 27/90
Progress: 28/90
Progress: 29/90
Progress: 30/90
Progress: 31/90
Progress: 32/90
Progress: 33/90
Progress: 34/90
Progress: 35/90
Progress: 36/90
Progress: 37/90
Progress: 38/90
Progress: 39/90
Progress: 40/90
Progress: 41/90
Progress: 42/90
Progress: 43/90
Progress: 44/90
Progress: 45/90
Progress: 46/90
Progress: 47/90
Progress: 48/90
Progress: 49/90
Progress: 50/90
Progress: 51/90
Progress: 52/90
Progress: 53/90


KeyboardInterrupt: 

In [ ]:
# Combine all data found in the 'data' directory into a single JSON file
combined_data = {}
for file_name in file_names:
    with open(f'data/{file_name}', 'r') as f:
        data = json.load(f)
        combined_data[file_name] = data
print(f'Combined data!')

with open(f'data/data_{wind_speed}ms.json', 'w') as f:
    json.dump(combined_data, f)
print(f'Saved combined data!')

# Remove individual data files
import os


In [ ]:
for file_name in file_names:
    os.remove(f'data/{file_name}')
print(f'Removed individual data files!')

In [ ]:
'''
# Plot the reflectances
import matplotlib.pyplot as plt

# Convert incoming zeniths to degrees
zenith_labels = np.rad2deg(outgoing_zeniths)

plt.plot(zenith_labels, [reflectances[zenith] for zenith in outgoing_zeniths])

plt.xlabel('Outgoing Zenith Angle')
plt.ylabel('Reflectance')
plt.title('Reflectance vs Outgoing Zenith Angle')
plt.show()
'''